In [ ]:
import pandas as pd
import ast
import requests
from bs4 import BeautifulSoup
# from IPython.core.display import display, HTML
pd.set_option("display.max_colwidth", None)

In [ ]:
TARGET_PATH = "./../../data/archived_snapshots_urls.csv"
recorded_urls = pd.read_csv(TARGET_PATH)
recorded_urls = recorded_urls[recorded_urls.archived_snapshots != '{}'].reset_index(drop=True)
recorded_urls['archived_snapshots'] = recorded_urls.archived_snapshots.apply(ast.literal_eval)
archived_snapshots_expanded = pd.json_normalize(recorded_urls.archived_snapshots)
recorded_urls = recorded_urls.drop(columns=["archived_snapshots"]).join(archived_snapshots_expanded)

In [ ]:
recorded_urls

In [ ]:
recorded_urls['closest.url']

In [ ]:
import requests

url = "https://web.archive.org/web/20221217083518/https://mastodon.xyz/about"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "de-DE,de;q=0.9,en;q=0.8",
    "Cache-Control": "no-cache",
    "Pragma": "no-cache",
}
response = requests.get(url, headers=headers, timeout=30, allow_redirects=True)
response.raise_for_status()

open("page.html", "w", encoding="utf-8").write(response.text)

In [ ]:
response.__dict__

In [ ]:
from bs4 import BeautifulSoup
import re

html = response.text.lower()
soup = BeautifulSoup(html, "html.parser")

out = {}

# "About" (best available in this HTML): meta description + title
title = soup.title.get_text(strip=True) if soup.title else None
meta_desc = soup.select_one('meta[name="description"]')
about = meta_desc["content"].strip() if meta_desc and meta_desc.has_attr("content") else None

out["About"] = {
    "title": title,
    "description": about,
}

# Helper: try to find a section by heading text (works only if content exists in HTML)
def extract_section_by_heading(heading_regex):
    heading = soup.find(
        lambda t: t.name in ["h1","h2","h3","h4","h5","h6"]
        and t.get_text(" ", strip=True)
        and re.search(heading_regex, t.get_text(" ", strip=True), re.I)
    )
    if not heading:
        return None

    # Collect text until the next heading
    parts = []
    for sib in heading.find_all_next():
        if sib.name in ["h1","h2","h3","h4","h5","h6"] and sib is not heading:
            break
        if sib.name in ["p","li","div","section"]:
            txt = sib.get_text(" ", strip=True)
            if txt:
                parts.append(txt)

    return "\n".join(parts).strip() or None

# Try to extract (will return None for your HTML)
out["Server Rules"] = extract_section_by_heading(r"(server\s*rules|regeln|rules)")
out["Moderated Servers"] = extract_section_by_heading(r"(moderated\s*servers|moderated|domain\s*blocks|blocklist|server\s*block)")

# Show whether the HTML even contains those phrases anywhere
full_text = soup.get_text(" ", strip=True).lower()
out["_contains_keywords"] = {
    "rules_word_present": "rules" in full_text or "regeln" in full_text,
    "moderated_word_present": "moderated" in full_text,
    "domain_blocks_word_present": "domain_blocks" in full_text,
}

print(out)


In [ ]:
out